In [ ]:
import pandas as pd

In [ ]:
import numpy as np

In [ ]:
import xarray as xr

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt

import seaborn as sns

In [ ]:
import matplotlib.animation as animation

In [ ]:
import research.src.stylesheets.style as style

In [ ]:
from ipywidgets.widgets import interact, IntSlider, FloatSlider, Layout

In [ ]:
annual_log_return_sigma = .7
sim_count = int(1e4)
sim_days = 365

In [ ]:
annual_simple_return = np.array([0, .1, .3, .5, 1])

In [ ]:
leverage = np.array([1, 2, 3, 4, 5, 7, 10, 20])

In [ ]:
annual_log_return = np.log(1 + annual_simple_return) - annual_log_return_sigma **2 / 2

In [ ]:
annual_simple_sigma = np.sqrt(
    (
        np.exp(annual_log_return_sigma ** 2) - 1
    )
    *
    np.exp(2 * annual_log_return + annual_log_return_sigma ** 2)
)


In [ ]:
ds = xr.Dataset()

ds = ds.assign_coords(leverage=leverage)

In [ ]:
np.random.seed(0)

ds["log_returns"] = xr.DataArray(
    data = np.random.normal(
        loc = annual_log_return / sim_days,
        scale = annual_log_return_sigma / np.sqrt(sim_days),
        size = (sim_count, sim_days, len(annual_log_return))
    ),
    coords = {
        "sim": np.arange(sim_count),
        "date": np.arange(sim_days),
        "annual_simple_return": annual_simple_return
    },
    dims = ["sim", "date", "annual_simple_return"]
).pipe(lambda x: xr.broadcast(x, ds["leverage"])[0])

ds = ds.transpose("leverage", "annual_simple_return", ...)

ds["annual_simple_sigma"] = xr.DataArray(
    data = annual_simple_sigma,
    coords = {"annual_simple_return": annual_simple_return},
    dims = ["annual_simple_return"]
)

In [ ]:
ds["cum_log_returns"] = (
    (
        ds["log_returns"]
    )
    .cumsum(dim="date")
)

ds["returns"] = (
    ds["log_returns"]
    .pipe(lambda x: np.exp(x) - 1)
)

ds["cum_returns"] = (
    ds["cum_log_returns"]
    .pipe(lambda x: np.exp(x) - 1)
)

ds["cum_min"] = xr.apply_ufunc(
    lambda x: np.minimum.accumulate(x, axis=0),
    ds["cum_returns"].transpose("date", ...),
)

ds["liquidated"] = (
    ds["cum_min"]
    <
    -(1. / ds["leverage"])
)

ds["fraction_liquidated"] = (
    ds["liquidated"]
    .pipe(lambda x: x.sum("sim") / x.count("sim"))
)

ds["fraction_liquidated_simwise"] = (
    ds["liquidated"]
    .pipe(lambda x: x.cumsum("sim") / (ds["sim"] + 1))
)

#### Viz

In [ ]:
style.use_stylesheet(["cms", "publication"])

In [ ]:
(
    ds["fraction_liquidated"]
    .sel(annual_simple_return=.3)
    .sel(leverage=5)
).T.to_pandas().plot(title="fraction of return paths liquidated over time", xlabel="days", ylabel="fraction of return paths")


In [ ]:
ax = sns.heatmap(
    (
        ds["fraction_liquidated"]
        .isel(date=-1)
        .rename({"annual_simple_return": "expected annual return"})
        .to_pandas()
    ),
    xticklabels=["{:.0%}".format(x) for x in ds["annual_simple_return"].values],
    annot=True,
    fmt=".0%",
    cmap="cms",
    cbar=False,
)

ax.set_title("liquidation likelihoods across leverage and alpha")



In [ ]:
# ax.get_figure().savefig("heatmap.jpg")

In [ ]:
(
    ds["cum_log_returns"]   
    .isel(date=-1)
    .sel(leverage=5)
    .sel(annual_simple_return = .3)
).to_pandas().plot.hist(bins=500)

In [ ]:
ax = (
    ds["cum_returns"]   
    .isel(date=-1)
    .sel(leverage=5)
    .sel(annual_simple_return = .3)
).to_pandas().plot.hist(bins=200, xlim=(-1, 5), title="simulated return distribution")

ax.set(xlabel="return", ylabel="sim count")

current_values = plt.gca().get_xticks()
ax.set_xticklabels(['{:,.0%}'.format(x) for x in current_values])

### Animation

In [ ]:
num_walks = 2000

In [ ]:
fig = plt.figure(figsize=(10, 5))

In [ ]:
ax = fig.add_subplot()

In [ ]:
ax2 = ax.twinx()

In [ ]:
ax.set(xlim=(0, 360), xlabel='days')
ax.set(ylim=(-1, 5), ylabel='ETH return')

current_values = ax.get_yticks()
_ = ax.set_yticklabels(['{:,.0%}'.format(x) for x in current_values])
    

In [ ]:
ax2.set(xlim=(0, 360), xlabel='days')
ax2.set(ylim=(0, 1), ylabel='% sims liquidated')

current_values = ax2.get_yticks()
_ = ax2.set_yticklabels(['{:,.0%}'.format(x) for x in current_values])

In [ ]:
liq_line = ax.plot(
    ds["date"].values,
    [-.2 for _ in ds["date"]],
    linestyle="--",
    color='#807E98',
    zorder=1,
)[0]

In [ ]:
frac_liq_line = ax2.plot([], [], color='#C8C7D8', zorder=2)[0]

In [ ]:
legend_lines = [
    ax.plot([], [], alpha = .5, color='#16B57F', zorder=0)[0],
    ax.plot([], [], alpha = .5, color='#E45555', zorder=0)[0],
]

In [ ]:
lines = [
    [
        ax.plot([], [], alpha = .05, color='#16B57F', zorder=0)[0],
        ax.plot([], [], alpha = .05, color='#E45555', zorder=0)[0],
    ]
    for _
    in range(num_walks)
]

In [ ]:
ax.legend(
    [legend_lines[0], legend_lines[1], liq_line, frac_liq_line],
    [
        "sims unliquidated",
        "sims liquidated",
        "liquidation threshold",
        "% sims liquidated (rhs)"
    ]
)

In [ ]:
def add_line(num):
    liquidated_mask=(
        ds["liquidated"]
        .sel(annual_simple_return=.3)
        .sel(leverage=5)
        .isel(sim=num)
        .shift(date=1)
    ) == True
    
    not_liquidated_mask=(
        (~ds["liquidated"])
        .sel(annual_simple_return=.3)
        .sel(leverage=5)
        .isel(sim=num)
    )
    
    lines[num][0].set_data(
        ds["date"].where(not_liquidated_mask, drop=True).values,
        (
            ds["cum_returns"]
            .sel(annual_simple_return=.3)
            .sel(leverage=5)
            .isel(sim=num)
            .where(not_liquidated_mask, drop=True)
            .values
        )
    )
    
    lines[num][1].set_data(
        ds["date"].where(liquidated_mask, drop=True).values,
        (
            ds["cum_returns"]
            .sel(annual_simple_return=.3)
            .sel(leverage=5)
            .isel(sim=num)
            .where(liquidated_mask, drop=True)
            .values
        )
    )
    
    frac_liq_line.set_data(
        ds["date"],
        (
            ds["fraction_liquidated_simwise"]
            .sel(annual_simple_return=.3)
            .sel(leverage=5)
            .isel(sim=num)
        )
    )
    
    return lines[num] + [liq_line] + [frac_liq_line]

In [ ]:
ani = animation.FuncAnimation(
    fig, add_line, num_walks, interval=100, blit=True)

In [ ]:
# %%timeit -r 1 -n 1

# ani.save("movie_16_9.mp4")

In [ ]:
for i in range(num_walks):
    add_line(i)

In [ ]:
# fig.savefig("sims_test.jpg", bbox_inches="tight")

In [ ]:
ax2.set(ylim=(0, .8))

ax.tick_params(bottom = False, left=False, right=False)
ax.axes.xaxis.set_visible(False)
ax.axes.yaxis.set_visible(False)
ax2.axes.yaxis.set_visible(False)
ax.legend().set_visible(False)
ax.set_title('Leverage Intuition', y=.5, x=.5, pad=-14, fontsize=22)


In [ ]:
fig

In [ ]:
fig.savefig("cover.jpg", bbox_inches="tight", dpi=500)

### Interactive Widget

In [ ]:
def simulate_and_plot(annual_simple_return, leverage):
    ds = simulate(
        annual_simple_return=np.array([annual_simple_return]),
        leverage=np.array([leverage])
    )
    
    (
        ds["fraction_liquidated"]
        .squeeze()
    ).T.to_pandas().plot(title="fraction of return paths liquidated over time", xlabel="days", ylabel="fraction of return paths", ylim=(0, 1))



In [ ]:
int_style = {'description_width': '150px'}
slider_layout = Layout(width='50%')

In [ ]:
interact(
    simulate_and_plot, 
    annual_simple_return=FloatSlider(
        min=-.5,
        max=3,
        step=.1,
        value=.3,
        description='Expected Annual Return (%)',
        style=int_style,
        layout=slider_layout
    ),
    leverage=FloatSlider(
        min=0,
        max=20,
        step=1,
        value=5,
        description='leverage',
        style=int_style,
        layout=slider_layout
    ),
);